### Below is the code for updating the Python Chatbot

In [ ]:
# Import Libraries
import json
import random
import os
import re
import csv
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
import datetime
import pandas as pd
import pandas as pd
import gensim
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from statistics import mode 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

Class Objects
*   Contexts - Here contexts are some flags which we switched on with each of the parameters or these can be used in conditional if-else statements. Below are the functions:

    - activate_context( ) and

    - deactivate_context( ) 

* We also have classes like 

    - FirstGreeting( ), 
    - IntentComplete( ) etc..

In [ ]:
class Context(object):

	def __init__(self,name):
		self.lifespan = 2
		self.name = name
		self.active = False

	def activate_context(self):
		self.active = True

	def deactivate_context(self):
		self.active = False

		def decrease_lifespan(self):
			self.lifespan -= 1
			if self.lifespan==0:
				self.deactivate_context()

class FirstGreeting(Context): 
	def __init__(self):
		self.lifespan = 1
		self.name = 'FirstGreeting'
		self.active = True

class IntentComplete(Context):
	def __init__(self):
		self.lifespan = 1
		self.name = 'IntentComplete'
		self.active = True


Intents - We have a class for Intent object. For every Intent we have it's name and particular action and parameters. For parameters we load a 'Parameter' object and it has name, prompts, placeholder etc...

In [ ]:
from abc import ABCMeta, abstractmethod
class Intent(object):	
	def __init__(self, name, params, action):
		self.name = name
		self.action = action
		self.params = []
		for param in params:
			self.params += [Parameter(param)]

class Parameter():
	def __init__(self, info):
		self.name = info['name']
		self.placeholder = info['placeholder']
		self.prompts = info['prompts']
		self.required = info['required']
		self.context = info['context']

In [ ]:
#!apt install unzip
!unzip chatbot_Zodoc.zip

Archive:  chatbot_Zodoc.zip
  inflating: chatbot_Zodoc/doctor_appointment.csv  
   creating: chatbot_Zodoc/entities/
  inflating: chatbot_Zodoc/entities/date.dat  
  inflating: chatbot_Zodoc/entities/day.dat  
  inflating: chatbot_Zodoc/entities/hospital.dat  
  inflating: chatbot_Zodoc/entities/location.dat  
  inflating: chatbot_Zodoc/entities/month.dat  
  inflating: chatbot_Zodoc/entities/speciality.dat  
  inflating: chatbot_Zodoc/entities/time.dat  
  inflating: chatbot_Zodoc/entities/year.dat  
   creating: chatbot_Zodoc/intents/
  inflating: chatbot_Zodoc/intents/get_Doctor_Appointment.dat  
  inflating: chatbot_Zodoc/intents/get_Zodiac_Sign.dat  
   creating: chatbot_Zodoc/params/
  inflating: chatbot_Zodoc/params/PARAM.cfg  
  inflating: chatbot_Zodoc/Zodiac_sign.csv  


In [ ]:
# YOUR CODE HERE for updating the path of the dat files
path_param = '/content/chatbot_Zodoc/params/PARAM.cfg'
path_intent= '/content/chatbot_Zodoc/intents/'
path_entities= '/content/chatbot_Zodoc/entities/'


In [ ]:
# YOUR CODE HERE for updating the path of csv file
path_doc_csv = '/content/chatbot_Zodoc/doctor_appointment.csv'
path_zod_csv = '/content/chatbot_Zodoc/Zodiac_sign.csv'

In [ ]:
df = pd.read_csv(path_doc_csv)
df

,Hospital,Speciality,Location,Start Date,End Date,Start Time,End Time
0,Aditya Hospitals,Cardiologist,Hyderabad,1/1/2020 9:00,8/1/2020 12:00,9:00,12:00
1,Aditya Hospitals,Cardiothoracic surgeon,Hyderabad,1/1/2020 10:00,8/1/2020 12:00,10:00,13:00
2,Aditya Hospitals,Endocrinologist,Hyderabad,1/1/2020 11:00,8/1/2020 14:00,11:00,14:00
3,Agadi Hospital and Research Centre,Gynaecologist,Bangalore,1/1/2020 12:00,8/1/2020 15:00,12:00,15:00
4,Agadi Hospital and Research Centre,Cardiologist,Bangalore,1/1/2020 13:00,8/1/2020 16:00,13:00,16:00
5,Agadi Hospital and Research Centre,Neurologist,Bangalore,1/1/2020 14:00,8/1/2020 17:00,14:00,17:00
6,Amar Leela Hospital,Endocrinologist,Delhi,1/1/2020 15:00,8/1/2020 18:00,15:00,18:00
7,Amar Leela Hospital,Cardiologist,Delhi,1/1/2020 16:00,8/1/2020 19:00,16:00,19:00
8,Sanjay Bansali Hospitals,Endocrinologist,Haryana,1/1/2020 17:00,8/1/2020 20:00,17:00,20:00
9,Sanjay Bansali Hospitals,ENT Specialist,Haryana,1/1/2020 18:00,8/1/2020 21:00,18:00,21:00


In [ ]:
#Define text representation method here
#load the data from .dat files and train your classifier
def load_data_And_trainModel():
  #YOUR CODE HERE for loading the data and training the classifier using any one of the representation (Counvectorizer, TF-IDF)
  lis = []
  labels = []
  uniques = []
  files = os.listdir(path_intent)
  for num,fil in enumerate(files):
    #print(fil)
    uniques.append(fil)
    if fil == ".ipynb_checkpoints":
      continue
    lines = open(path_intent+fil).readlines()
    for i, line in enumerate(lines):
      # print(i)
      # print(line)
      # print("___________")
      labels.append(num)
      lis.append(line)
  vectorizer = TfidfVectorizer()
  vectorizer.fit(lis)
  X = vectorizer.transform(lis).toarray()
  # label1 = [0]*37
  # label2 = [1]*48
  # labels = label1 + label2
  neigh = KNeighborsClassifier(n_neighbors=3)
  neigh.fit(X,labels)
  return neigh,vectorizer,uniques

In [ ]:
model,vec,uniq = load_data_And_trainModel()

In [ ]:
lis = []
labels = []
uniques = []
files = os.listdir(path_intent)
for num,fil in enumerate(files):
  print(num)
  print("___________")
  print(fil)
  uniques.append(fil)
  if fil == ".ipynb_checkpoints":
    continue
  lines = open(path_intent+fil).readlines()
  for i, line in enumerate(lines):
    print(i)
    print(line)
    print("_______________________")
    labels.append(num)
    lis.append(line)
vectorizer = TfidfVectorizer()
vectorizer.fit(lis)
X = vectorizer.transform(lis).toarray()
# label1 = [0]*37
# label2 = [1]*48
# labels = label1 + label2
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X,labels)

In [ ]:
labels

In [ ]:
X.shape

(92, 63)

In [ ]:
# Take the user input as test data and predict using the above classifier. Get the mode of the predicted values.
def intentPredict(user_input):
   #usr = list(user_input)
   model,vectorizer,uniques = load_data_And_trainModel()# YOUR CODE HERE for the prediction
   vc = vectorizer.transform([user_input]).toarray()
   c = model.predict(vc)
   foo = uniques[int(c)]
   foo = foo[:-4]
   return foo

In [ ]:
l = intentPredict('I need the doctor appointment')

In [ ]:
l

'get_Doctor_Appointment'

Defining functions for Loading Intent, Collecting params, Checking actions, Identifying Intents and Getting Attributes 

In [ ]:
def loadIntent(path, intent):
    #print(path)
    #print(intent)
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    for para in current_intent.params:     	
        if para.required:
            if para.name not in attributes:
                return random.choice(para.prompts), context
    return None, context
   
def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent as mentioned 
    in the intent config file. Performs actions pertaining to current intent'''
    context = IntentComplete()
    return current_intent.action, context

def input_processor(user_input, context, attributes, intent):
    '''Update the attributes, abstract over the entities in user input'''
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    return attributes, cleaned_input
 
def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    if(current_intent==None):
        return loadIntent(path_param,intentPredict(clean_input)) #Calling the intentPredict
    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''

    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        files = os.listdir(path_entities)
        entities = {}
        for fil in files:
            if fil == ".ipynb_checkpoints":
                continue
            lines = open(path_entities+fil).readlines()
            for i, line in enumerate(lines):
                if lines[i].endswith('/n'):
                  lines[i] = line[:-2]
                lines[i]=lines[i].strip()

                if lines[i].lower() in uinput.lower():
                    attributes[fil[:-4]] = lines[i]
                uinput = re.sub(line,r'$'+fil[:-4],uinput,flags=re.IGNORECASE)  # Identifying entities values from the input
        return attributes, uinput            

In [ ]:
class Session:
    
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):     
        '''Initialise a default session'''
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
  
        #Contexts are flags which control dialogue flow       
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        self.current_intent = FirstGreeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()
                
    global intentCompleted
    def reply(self, user_input):
        '''Generate response to user input'''
        global intentCompleted
        intentCompleted=False
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)

        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)

        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)
        global attr
        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
                if prompt.endswith('()'):
                  attr=self.attributes
                  prompt=eval(prompt)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            intentCompleted=True # It becomes true when the intent is completed
            self.context = FirstGreeting()
            self.current_intent = None
        return prompt

In [ ]:
# Define the method with the action name mentioned in the newparams.cfg file

def doctorAppointment_Action(): #function name should be your action name in newparams.cfg file
    global attr
    print("User given attributes", attr)
    date = attr['date']#YOUR CODE HERE for comparing and returning the final output
    time = int(attr['time'])
    hospital = attr['hospital']
    speciality = attr['speciality']
    location = attr['location']
    usr = datetime.datetime.strptime(date,'%d/%m/%Y %H:%M')

    avail = False
    with open(path_doc_csv, 'r') as file:
      reader = csv.reader(file)
      next(reader)
      for row in reader: 
        strt = datetime.datetime.strptime(row[3],'%d/%m/%Y %H:%M')
        ed = datetime.datetime.strptime(row[4],'%d/%m/%Y %H:%M')
        
        #print(row[0],row[1])       
        if (row[0].lower() in hospital.lower() and row[1].lower() in speciality.lower() and row[2].lower() in location.lower()):
          if (strt<=usr<=ed):
            avail = True
      if avail:
        s = 'The doctor is availble for    ' + date + '    for   ' + speciality + '    in    ' + hospital + '    at    ' + location
      if avail==False:
        s = 'The doctor is not availble for    ' + date + '    for    ' + speciality + '    in    ' + hospital + '    at    ' + location
    return s

**Below function is for 
Zodiac_Sign intent**

In [ ]:
def zodiacSign_Action():    
  global attr
  print("choosen attributes are",attr)
  year = attr['year']
  month = attr['month'] # month is a string, and we need to convert it to a month index
  day = int(attr['day'])
  
  try:
      month = int(datetime.datetime.strptime(month,'%b').strftime('%m'))
  except:
      month = int(datetime.datetime.strptime(month,'%B').strftime('%m'))
  
  zodiac_sign=""
  usr_dob = (month,day)
  with open(path_zod_csv, 'r') as file:
    reader = csv.reader(file)
    for row in reader:        
        if filter(row[0]) <= usr_dob <= filter(row[1]):
            zodiac_sign=row[2]
  return "Your zodiac sign is "+zodiac_sign
   

def filter(X):
    date = X.split()
    try:
      month = int(datetime.datetime.strptime(date[0],'%b').strftime('%m'))
    except:
      month = int(datetime.datetime.strptime(date[0],'%B').strftime('%m'))
    day = int(datetime.datetime.strptime(date[1],'%d').strftime('%d'))
    return (month,day)

In [ ]:
class BOT_config():
  def __init__(self):
    self.welcome='BOT: Hi! Welcome to Talentsprint Hackathon, How may i assist you?'
    self.greet = ["BOT: Good Morning, Team 8","BOT: Good Evening, Team 8","BOT: Good Afternoon, Team 8"]
    self.confirm="BOT: Do you want to proceed (Y/N)? "
    self.welcome_back="BOT: welcome back.., How can I help you?"
    self.bye="Bye, hope to meet again"
    self.exits=["finish","exit","end"]

In [ ]:
GREETING_INPUTS = ["Good Morning","Good Evening","Good Afternoon"]
print(BOT_config().welcome)
session=Session()
attr={}
intentCompleted=False
while (True):
    inp = input('User: ')
    for i in range(len(GREETING_INPUTS)):
      if inp == GREETING_INPUTS[i]:
        print(BOT_config().greet[i])
    if inp in BOT_config().exits:
      break 
    prompt=session.reply(inp)
    print ('BOT:', prompt)
    if intentCompleted:
      a=input(BOT_config().confirm)
      if a.lower().startswith( "y"):
        no_Response=[]
        print(BOT_config().welcome_back)           
      else:
        print(BOT_config().bye)
        break 

BOT: Hi! Welcome to Talentsprint Hackathon, How may i assist you?
User: I want to know my birth sign
BOT: Which day(date) were you born?
User: 7 may
BOT: What is your year of birth?
User: 2000
choosen attributes are {'month': 'May', 'time': '20', 'day': '20', 'year': '2000'}
BOT: Your zodiac sign is Taurus
BOT: Do you want to proceed (Y/N)? Y
BOT: welcome back.., How can I help you?
User: my date of birth is on 25 july 1997
choosen attributes are {'year': '1997', 'month': 'Jul', 'time': '19', 'day': '25'}
BOT: Your zodiac sign is Leo
BOT: Do you want to proceed (Y/N)? Y
BOT: welcome back.., How can I help you?
User: suggest a specialist who has a good review
BOT: which day(date) would you like to meet the doctor?
User: exit


In [ ]:
df

,Hospital,Speciality,Location,Start Date,End Date,Start Time,End Time
0,Aditya Hospitals,Cardiologist,Hyderabad,1/1/2020 9:00,8/1/2020 12:00,9:00,12:00
1,Aditya Hospitals,Cardiothoracic surgeon,Hyderabad,1/1/2020 10:00,8/1/2020 12:00,10:00,13:00
2,Aditya Hospitals,Endocrinologist,Hyderabad,1/1/2020 11:00,8/1/2020 14:00,11:00,14:00
3,Agadi Hospital and Research Centre,Gynaecologist,Bangalore,1/1/2020 12:00,8/1/2020 15:00,12:00,15:00
4,Agadi Hospital and Research Centre,Cardiologist,Bangalore,1/1/2020 13:00,8/1/2020 16:00,13:00,16:00
5,Agadi Hospital and Research Centre,Neurologist,Bangalore,1/1/2020 14:00,8/1/2020 17:00,14:00,17:00
6,Amar Leela Hospital,Endocrinologist,Delhi,1/1/2020 15:00,8/1/2020 18:00,15:00,18:00
7,Amar Leela Hospital,Cardiologist,Delhi,1/1/2020 16:00,8/1/2020 19:00,16:00,19:00
8,Sanjay Bansali Hospitals,Endocrinologist,Haryana,1/1/2020 17:00,8/1/2020 20:00,17:00,20:00
9,Sanjay Bansali Hospitals,ENT Specialist,Haryana,1/1/2020 18:00,8/1/2020 21:00,18:00,21:00


In [ ]:
path_cfg = '/content/chatbot_Zodoc/params/PARAM.cfg'
with open(path_cfg) as fil:
  dat = json.load(fil)

In [ ]:
GREETING_INPUTS = ["Good Morning","Good Evening","Good Afternoon"]
GREETING_RESPONSES = ["Good Morning","Good Evening","Good Afternoon"]
           
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

self.greet = ["Good Morning","How are you?","Good Afternoon","Its great to start a aconversation with you"]
 if inp in BOT_config().greet:
      print()
      continue
